# Загрузка библиотек

In [2]:
import pandas as pd
import os
import numpy as np
from ultralytics import YOLO
import random
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from datetime import datetime
from pathlib import Path
from IPython.display import clear_output

## Фиксирование seed'а

In [3]:
RANDOM_STATE = 42

seed = RANDOM_STATE
np.random.seed(seed)
random.seed(seed)
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)

# Подготовка датасета для валидации

In [11]:
df_test = pd.read_pickle('../data/common_files/df_markup.pkl')
df_test = df_test.loc[
    (df_test['terminal']==1) &
    (df_test['file_name'].apply(lambda x: 'augm' not in x)) 
]
df_test.head()

,file_name,quality_photo,terminal,receipt,terminal_damaged,terminal_undamaged,terminal_unrecognized_defect,other,anticlass,hash
15,..\data\sorted_data_merged\undamaged\╨Ч╨Э╨Ю030...,1,1,0,0,1,0,0,0,e71786ba592aadd9f340f424eab8576cfefd78a0b2b181...
18,..\data\sorted_data_merged\receipt_undamaged\З...,1,1,1,0,1,0,0,0,0d05484d6eff63916387e1641987a398403ca557fde403...
21,..\data\sorted_data_merged\undamaged\╨Ч╨Э╨Ю030...,1,1,0,0,1,0,0,0,051d2d73f9a5535533771d190d6c00b5e3f2b161de3bc0...
24,..\data\sorted_data_merged\undamaged\ЗНО030141...,1,1,0,0,1,0,0,0,ada2e148279c8421c69a6a874f05b0fd01bbe1f5c708e9...
47,..\data\sorted_data_merged\damaged\╨Ч╨Э╨Ю03017...,1,1,0,1,0,0,0,0,3b54f90fdffae40e9ade93b2c3ecd0230ce2e64faa57a9...


# Формирование прогнозов

In [16]:
model_terminal = YOLO('../terminal_object_detection/best.pt')
model_quality = YOLO('last_defect_od_crop.pt')

In [42]:
val_folder = Path('val_photos3')
if not os.path.exists(val_folder):
    os.makedirs(val_folder)
for image_data in tqdm(df_test.sample(200).iterrows()):
    image_path = image_data[1]['file_name']
    damaged_flag = image_data[1]['terminal_damaged']
    start = datetime.now()
    im = Image.open(image_path)
    cropped_terminals = model_terminal.predict(im, imgsz=640)
    n_terminals = len(cropped_terminals[0].boxes)
    if not n_terminals:
        file_name = val_folder / image_path.split('\\')[-1]
        plt.imshow(im)
        plt.title('Terminal is not detected')
        plt.savefig(file_name)
        plt.close()
        continue
    fig, ax = plt.subplots(1, n_terminals+1, figsize=(19, 10))
    ax[0].imshow(im)
    for i, box in enumerate(cropped_terminals[0].boxes):
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        terminal_box = im.crop((x1, y1, x2, y2))
        ax[i+1].imshow(terminal_box)
        defect_results = model_quality.predict(terminal_box, imgsz=1280)
        for j, box in enumerate(defect_results[0].boxes):
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            cls = int(box.cls)
            conf = float(box.conf)
            rect = plt.Rectangle(
                (x1, y1), x2 - x1, y2 - y1, fill=False, 
                linewidth=2, edgecolor='red'
            )
            ax[i+1].add_patch(rect)
            ax[i+1].text(x1, y1 - 10, f'{defect_results[0].names[int(cls)]} {(conf):.2f}', color='white', fontsize=9,
                bbox=dict(facecolor='red', alpha=0.5))
    end = datetime.now()
    print(f'Time: {end-start}')
    damage_status = 'Terminal is not damaged'
    if damaged_flag:
        damage_status = 'Terminal is damaged'
    ax[0].set_title(f"Original Image\n{damage_status}")
    ax[0].set_axis_off()
    ax[i+1].set_title('Damage Detection')
    ax[i+1].set_axis_off()
    file_name = val_folder / image_path.split('\\')[-1]
    plt.savefig(file_name.__str__().replace('heic', 'jpg'))
    plt.close(fig)
clear_output()